In [1]:
import json
import os

In [2]:
with open('gnpslibraryjson_filtered.json', 'rb') as f:
    data = json.load(f)

In [3]:
len(data)

77253

In [4]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry


In [10]:
def check_spectrum_lines(spectrum):
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
            
        if l.startswith('CHARGE') and l.strip().endswith('-1+'):
            return False
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename):
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            if line.startswith('END ION'):
                # Check if we have structural info
                if check_spectrum_lines(spectrum):
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
    return matched_spectra

path = '/home/grimur/iokr/data/mibig/gnps-mibig/gnps_libraries_20190726/ccms-data/uploads/speclibs/'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f))
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))
        
        #fn = '/home/grimur/gnps-mibig/gnps_libraries_20190726/ccms-data/uploads/speclibs/GNPS-LIBRARY/GNPS-LIBRARY.mgf'
        
print('write')

#with open('/home/grimur/gnps-mibig/matched.mgf', 'w') as f:
with open('matched.mgf', 'w') as f:
    for c in collected_spectra:
        for l in c:
            try:
                f.write(l)
            except:
                print(l)
                raise
        #f.write('\n')

('GNPS-LIBRARY.mgf', 2139)
('GNPS-NIH-CLINICALCOLLECTION2.mgf', 2147)
('MASSBANKEU.mgf', 3451)
('LDB_NEGATIVE.mgf', 3451)
('MMV_POSITIVE.mgf', 3451)
('CASMI.mgf', 3973)
('GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf', 3973)
('GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf', 5433)
('LDB_POSITIVE.mgf', 5433)
('MMV_NEGATIVE.mgf', 5433)
('GNPS-SELLECKCHEM-FDA-PART1.mgf', 5480)
('BILELIB19.mgf', 5480)
('GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf', 6133)
('GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf', 7400)
('DEREPLICATOR_IDENTIFIED_LIBRARY.mgf', 7400)
('MASSBANK.mgf', 18926)
('SUMNER.mgf', 18926)
('MONA.mgf', 68167)
('GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf', 68243)
('GNPS-FAULKNERLEGACY.mgf', 68244)
('MIADB.mgf', 68244)
('GNPS-COLLECTIONS-MISC.mgf', 68269)
('HMDB.mgf', 70504)
('GNPS-PRESTWICKPHYTOCHEM.mgf', 70647)
('GNPS-EMBL-MCF.mgf', 71135)
('GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf', 71135)
('RESPECT.mgf', 76000)
('PNNL-LIPIDS-POSITIVE.mgf', 76000)
('PNNL-LIPIDS-NEGATIVE.mgf

UnicodeEncodeError: 'ascii' codec can't encode character u'\ufffd' in position 114: ordinal not in range(128)

In [20]:
from pyteomics import mgf

In [66]:
cnt = 0
print('load')
d = mgf.read('ALL_GNPS.mgf', use_index=False)


load


In [61]:
print(data[0]['GNPSID'])
print(m['params']['spectrumid'])

gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry


CCMSLIB00000001548
CCMSLIB00000072188


In [70]:
spectra_with_structure = []
for spectrum in mgf.read('ALL_GNPS.mgf', use_index=False):
    spectrum_id = spectrum['params']['spectrumid']
    if spectrum_id in gnps_data:
        spectrum['params']['inchikey'] = gnps_data[spectrum_id]['COMPOUND_INCHIKEY']
        spectrum['params']['smiles'] = gnps_data[spectrum_id]['COMPOUND_SMILES']
        spectrum['params']['inchi'] = gnps_data[spectrum_id]['COMPOUND_INCHI']
        spectrum['params']['libraryquality'] = gnps_data[spectrum_id]['LIBRARY_QUALITY']
        spectrum['params']['title'] = gnps_data[spectrum_id]['COMPOUND_NAME']
        spectra_with_structure.append(spectrum)


In [71]:
len(spectra_with_structure)

5079

In [75]:
import os
spec_w_struct = []
libpath = '/home/grimur/gnps-mibig/gnps_libraries_20190726/ccms-data/uploads/speclibs'
for d in os.listdir(libpath):
    for f in os.listdir(os.path.join(libpath, d)):
        if not f.endswith('.mgf'):
            continue
        print(os.path.join(d,f))
        for spectrum in mgf.read(os.path.join(libpath, d, f), use_index=False):
            spectrum_id = spectrum['params']['spectrumid']
            if spectrum_id in gnps_data:
                spectrum['params']['inchikey'] = gnps_data[spectrum_id]['COMPOUND_INCHIKEY']
                spectrum['params']['smiles'] = gnps_data[spectrum_id]['COMPOUND_SMILES']
                spectrum['params']['inchi'] = gnps_data[spectrum_id]['COMPOUND_INCHI']
                spectrum['params']['libraryquality'] = gnps_data[spectrum_id]['LIBRARY_QUALITY']
                spectrum['params']['title'] = gnps_data[spectrum_id]['COMPOUND_NAME']
                spec_w_struct.append(spectrum)



GNPS-LIBRARY/GNPS-LIBRARY.mgf


PyteomicsError: Pyteomics error, message: "invalid literal for int() with base 10: '-1+'"

In [79]:
import sys
sys.path.append("/home/grimur/git/nplinker/nplinker/prototype/")
print(sys.path)


['/home/grimur/gnps-mibig', '/home/grimur/git/ghlib', '/home/grimur/gnps-mibig', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python37.zip', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/lib-dynload', '', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/pandas-0.24.2-py3.7-linux-x86_64.egg', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/PATRIC_Tools-1.1.0-py3.7.egg', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/ply-3.11-py3.7.egg', '/home/grimur/miniconda2/envs/jupyter-py3/lib/python3.7/site-packages/IPython/extensions', '/home/grimur/.ipython', '/home/grimur/git/nplinker/nplinker/prototype/']


In [80]:
import parsers

In [89]:
gnps_data

{'CCMSLIB00000001548': {'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
  'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
  'LIBRARY_QUALITY': '1',
  'GNPSID': 'CCMSLIB00000001548',
  'COMPOUND_NAME': 'Hoiamide B',
  'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'},
 'CCMSLIB00000001552': {'COMPOUND_INCHIKEY': 'USXIYWCPCGVOKF-NOENWEJRSA-N',
  'COMPOUND_INCHI': 'InChI=1S/C27H34Cl2N2O9S2/c1-13-17(9-8-10-27(7,28)29)38-23(34)15-11-42-21(30-15)19(37-14(2)32)26(5,6)40-24(35)16-12-41-20(31-16)18(25(3,4)36)39-

In [11]:
def check_spectrum_lines(spectrum):
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
            
        if l.startswith('CHARGE') and l.strip().endswith('-1+'):
            return False
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename):
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            if line.startswith('END ION'):
                # Check if we have structural info
                if check_spectrum_lines(spectrum):
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
    return matched_spectra

#path = '/home/grimur/gnps-mibig/gnps_libraries_20190726/ccms-data/uploads/speclibs/'
path = '/home/grimur/iokr/data/mibig/gnps-mibig/gnps_libraries_20190726/ccms-data/uploads/speclibs/'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f))
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))
        
        #fn = '/home/grimur/gnps-mibig/gnps_libraries_20190726/ccms-data/uploads/speclibs/GNPS-LIBRARY/GNPS-LIBRARY.mgf'
        
print('write')

#with open('/home/grimur/gnps-mibig/matched.mgf', 'w') as f:
with open('matched.mgf', 'w') as f:
    for c in collected_spectra:
        for l in c:
            f.write(l)
        #f.write('\n')

('GNPS-LIBRARY.mgf', 2139)
('GNPS-NIH-CLINICALCOLLECTION2.mgf', 2147)
('MASSBANKEU.mgf', 3451)
('LDB_NEGATIVE.mgf', 3451)
('MMV_POSITIVE.mgf', 3451)
('CASMI.mgf', 3973)
('GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf', 3973)
('GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf', 5433)
('LDB_POSITIVE.mgf', 5433)
('MMV_NEGATIVE.mgf', 5433)
('GNPS-SELLECKCHEM-FDA-PART1.mgf', 5480)
('BILELIB19.mgf', 5480)
('GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf', 6133)
('GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf', 7400)
('DEREPLICATOR_IDENTIFIED_LIBRARY.mgf', 7400)
('MASSBANK.mgf', 18926)
('SUMNER.mgf', 18926)
('MONA.mgf', 68167)
('GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf', 68243)
('GNPS-FAULKNERLEGACY.mgf', 68244)
('MIADB.mgf', 68244)
('GNPS-COLLECTIONS-MISC.mgf', 68269)
('HMDB.mgf', 70504)
('GNPS-PRESTWICKPHYTOCHEM.mgf', 70647)
('GNPS-EMBL-MCF.mgf', 71135)
('GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf', 71135)
('RESPECT.mgf', 76000)
('PNNL-LIPIDS-POSITIVE.mgf', 76000)
('PNNL-LIPIDS-NEGATIVE.mgf

UnicodeEncodeError: 'ascii' codec can't encode character u'\ufffd' in position 114: ordinal not in range(128)

In [128]:
smiles_set = set([x[1]['COMPOUND_SMILES'] for x in gnps_data.items()])

In [130]:
for i in smiles_set:
    print(i)

O=C1C2=C(O)C=C(O)C=C2OC(C3=CC(O)=C(O)C=C3)=C1O[C@H]4[C@H](OC(/C=C/C5=CC=C(O)C=C5)=O)[C@@H](O[C@@H]6OC[C@H](O)[C@H](O)[C@H]6O)[C@H](O)[C@@H](CO[C@H]7[C@H](O)[C@H](O)[C@@H](O)[C@H](C)O7)O4
[H][C@@](COP(O)(OCCN)=O)(OC(CCCCCCCCCCCCCCC)=O)COC(CCCCCCCCCCCCCC)=O
Cn1c2c(c(=O)n(c1=O)C)[nH]cn2
Clc1ccc2c(CCc3cccnc3C2=C2CCNCC2)c1
C1(=CC=C(C=C1)NCC2N(C3=C(N=2)C=C(C=C3)C(N(CCC(OCC)=O)C4=CC=CC=N4)=O)C)/C(=N\C(OCCCCCC)=O)N
Fc1ccc(cc1)N(C(C)C)C(=O)C(O)=O
[H][C@@]12CCC(C(C)=O)[C@@]1(C)C[C@@H](O)[C@@]1([H])[C@@]2([H])CCC2=CC(=O)CC[C@]12C
OC(C1=CC=C(CCCCCCCCCCCCCCC)C=C1OC(C)=O)=O
CCCCCCCCCC(=O)OCC(O)COP(O)(=O)OCC[N+1](C)(C)C
O=C(OC)CCCCCCc1ccc(OCCOCCO)cc1
OC[C@@H]1[C@@H](O)[C@H](O)[C@@H](O)[C@H](C2=C(O)C3=C(C=C2O)OC4=CC(O)=C(O)C=C4C3=O)O1
CC1=C(C=C(C=C1)N=C=O)N=C=O
C(C(C(=O)O)N)SCC(=O)O
C(C(C(C(=O)COP(=O)(O)O)O)O)OP(=O)(O)O
CCCCCCCCCCCCC/C=C/[C@H]([C@H](CO)N)O
c1ccc2nc3ccccc3nc2c1
CC(C(C(=O)NC=C1CC(C(O1)N2CCC(=O)NC2=O)O)NC(=O)C(CCSC)NC(=O)NC(CC3=CC(=CC=C3)O)C(=O)O)N(C)C(=O)C4CC5=C(CN4)C=CC(=C5)O
NC(C(NC(C

SC1=NCCC12C(=Nc3ccccc32)S
CCC1=CC=C(CCOC2=CC=C(CC3SC(=O)NC3=O)C=C2)N=C1
O=C1C2=CC=C(O[C@H]3[C@H](O)[C@@H](O)[C@H](O)[C@@H](CO)O3)C(O)=C2OC(C4=CC(O)=C(OC)C=C4)C1
OC(CCN1CCCCC1)(C2C[C@H]3C[C@@H]2C=C3)c4ccccc4.Cl
CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc2c[nH]c3ccccc23)NC(=O)C(Cc4c[nH]cn4)NC(=O)C5CCC(=O)N5)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N6CCCC6C(=O)NNC(=O)N
CCC(C)C(N)CC(O)=O
CC[C@H](C)C(=O)O[C@H]1C[C@H](O)C=C2C=C[C@H](C)[C@H](CC[C@@H](O)C[C@@H](O)CC(O)=O)[C@@H]12
CC(C)(C)C1=CC=C(C=C1)C(=O)CC(=O)C2=CC=C(C=C2)OC
COc1ccc2[nH]c3c(CCN4C(=O)N([C@@H](C)C(=O)NCc5ccccc5OC)C(=O)[C@@]43C)c2c1
Cc1ccc(=O)n(c1)c2ccccc2
C[C@H]1C[C@H]2[C@@H]3CC[C@H](C(=O)C)[C@@]3(C)C[C@H](O)[C@@H]2[C@@]4(C)CCC(=O)C=C14
CC(C)(C(=O)O)c1ccc(cc1)C(O)CCCN2CCC(CC2)C(O)(c3ccccc3)c4ccccc4.Cl
C1CC2=C(C1)C(=O)OC3=C4CCCN5C4=C(CCC5)C=C23
CC(C4)(C(CC(O)C4)=3)C(C2)C(CC3)C(C1)C(C)(C2)C(C(C)CCCC(C)C)C1
CCCCCCCCCCCCCCCC[N+](C)(C)CCN(Cc1ccc(OC)cc1)c2ncccn2.[Br-]
CC(C)c1ccc(C)c(C(=O)O)c1O
COC1=CC(=C2C(=O)CC(OC2=C1

COc1cc(O)cc2OC[C@@H]3[C@@H](Oc4c3ccc(O)c4CC=C(C)C)c12
CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2Cc3ccccc3C[C@H]2C(=O)O.Cl
(CC(=O)O3)O)C
OC(=O)C(N)CCCC(N)C(O)=O
OC1=C(O)C=C(C2=C(O[C@H]3OC[C@H](O)[C@@H](O)[C@@H]3O)C(C4=C(O)C=C(O)C=C4O2)=O)C=C1
OC(=O)\C=C\C1=CC=CC=C1
CCC(=O)OCC(=O)C1(OC(=O)CC)[C@H](C)C[C@H]2[C@@H]3[C@H](Cl)CC4=CC(=O)C=C[C@]4(C)[C@H]3[C@@H](O)C[C@@]21C
C[C@@H](O)/C=C(C)/CCC/C(C)=C/CC(OC(OC)=C1C)=C(C)C1=O
COc1cc(/C=C/C(=O)O)cc(OC)c1OC
CC(Oc1ccc2c3CCCc3c(=O)oc2c1C)C(=O)N4C[C@H]5C[C@@H](C4)c6cccc(=O)n6C5
NCC(=O)N[C@@H](CC(C)C)C(O)=O
CC(CSSCC(C)C(=O)N1CCCC1C(=O)O)C(=O)N2CCCC2C(=O)O
COc1cc(cc(OC)c1OC)C2OCC3C(OCC23)c4cc(OC)c(OC)c(OC)c4
CCc1cc(=O)oc2c3CCC(C)(C)Oc3cc(OCC(=O)NCc4ccco4)c12
OCC(CO)CCn(c2)c(N1)c(n2)C(=O)N=C(N)1
CC1=NC=C(C(=C1O)CO)CO
CCCCCCCCCCCCCCCC(=O)O[C@H](CC([O-])=O)C[N+](C)(C)C
OC(=O)CCC(N)C(=O)NC(CCC(O)=O)C(O)=O
CC(=CC(=O)N(CCCC1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NCC(=O)N1)CO)CO)CCCN(C(=O)C=C(C)CC(=O)O)O)CCCN(C(=O)C=C(C)CC(=O)O)O)O)CC(=O)O
NC1=CC(=O)N=C(N)N1
C

In [131]:
gnps_data['CCMSLIB00000072151']

{'COMPOUND_INCHIKEY': 'ZNMZYHOEYPCVMP-MITQYPHISA-N',
 'COMPOUND_INCHI': 'InChI=1S/C43H84NO8P/c1-3-5-7-9-11-13-15-17-18-19-20-21-22-24-25-27-29-31-33-35-42(45)49-39-41(40-51-53(47,48)50-38-37-44)52-43(46)36-34-32-30-28-26-23-16-14-12-10-8-6-4-2/h14,16,41H,3-13,15,17-40,44H2,1-2H3,(H,47,48)/b16-14-/t41-/m1/s1',
 'LIBRARY_QUALITY': '3',
 'GNPSID': 'CCMSLIB00000072151',
 'COMPOUND_NAME': 'Phosphatidylethanolamine (22:0/16:1) Abbr: BPoPE',
 'COMPOUND_SMILES': 'N/A'}